
## Building a Kalman Filter

In this notebook, we'll create a Kalman Filter using filterpy library. We will also code our own Kalman Filter.

To make it easier, we'll use a library called Filterpy. The filterpy library can implement a full filter, but we'll also code it ourselves for better understanding.

If you find some sentences similar to this Kalman Filter Book, it's because I took some matrices and concepts from this amazing book that I highly recommend!

Anyway, let's worry about tracking Speedy, our self-driving car, in 1, and 2D, using Kalman Filters!

In this notebook, you'll learn how to predict Speedy's future positions and create a real time Machine!


# Part 1 - First Order Kalman Filter

The first part of our exercise will be to implement what's called a First Order Kalman Filter. Let's not confuse you with details right now, we'll go back to orders later.

 Step 1 - Let's call the filterpy library and track Speedy in 1D

We will start by tracking Speedy in X direction.<p>
That's it!<p>
We need to carefully design our matrices to know what we're talking about.

from filterpy.kalman import KalmanFilter
from scipy.linalg import block_diag
from filterpy.common import Q_discrete_white_noise
import numpy as np

In [1]:
from filterpy.kalman import KalmanFilter
from scipy.linalg import block_diag
from filterpy.common import Q_discrete_white_noise
import numpy as np

We will track position and velocity, so the state variable needs both of these. <p>The matrix formulation is:

$$ \mathbf x = \begin{bmatrix}x\\\dot x\end{bmatrix}$$

So now we have to design our state transition. The Newtonian equations for a time step are:

$$\begin{aligned} x_t &= x_{t-1} + v\Delta t \\
 v_t &= v_{t-1}\end{aligned}$$
 
Recall that we need to convert this into the linear equation

$$\begin{bmatrix}x\\\dot x\end{bmatrix} = \mathbf F\begin{bmatrix}x\\\dot x\end{bmatrix}$$

Setting

$$\mathbf F = \begin{bmatrix}1 &\Delta t\\ 0 & 1\end{bmatrix}$$

gives us the equations above and implements what's called the **Constant Velocity Model**.

Finally, we design the measurement function. The measurement function needs to implement

$$\mathbf z = \mathbf{Hx}$$

Our sensor still only reads position, so it should take the position from the state, and 0 out the velocity and acceleration, like so:

$$\mathbf H = \begin{bmatrix}1 & 0 \end{bmatrix}$$

While the provided code sets up and initializes the Kalman Filter, additional steps are typically required to fully configure the filter for practical use:

    Setting the Initial State Covariance Matrix (P): Defines the initial uncertainty in the state estimate.
    Defining the State Transition Matrix (F): Describes how the state evolves from one time step to the next.
    Defining the Process Noise Covariance (Q): Quantifies the uncertainty in the process model.
    Defining the Measurement Function (H): Relates the state to the measurement.
    Defining the Measurement Noise Covariance (R): Represents the expected noise in the measurements.

In [2]:
def OneDimensionKF(R_std, Q_std, dt):
    """ Create first order 2D Kalman filter."""
    kf = KalmanFilter(dim_x=2, dim_z=1)
    kf.x = np.zeros(2)
    kf.P = np.eye(2)*500
    kf.R = np.eye(1)*R_std
    kf.Q = Q_discrete_white_noise(dim = 2, dt = dt, var = Q_std)
    kf.F = np.array([[1., dt],
                     [0., 1]])
    kf.H = np.array([[1., 0]])
    return kf

In [3]:
measurements = [1, 2, 3, 4, 5]
Q_std = 0.01
R_std = 5
dt = 1.

kf = OneDimensionKF(R_std, Q_std, dt)
for z in measurements:
    kf.predict()
    print("Predict...")
    print(kf.x)
    #print(kf.P)
    print("Measured: ",z)
    print("Update...")
    kf.update(z)
    print(kf.x)
    #print(kf.P)

Predict...
[0. 0.]
Measured:  1
Update...
[0.99502489 0.49751618]
Predict...
[1.49254106 0.49751618]
Measured:  2
Update...
[1.99046854 0.98122233]
Predict...
[2.97169088 0.98122233]
Measured:  3
Update...
[2.99509117 0.9951294 ]
Predict...
[3.99022057 0.9951294 ]
Measured:  4
Update...
[3.99703539 0.99803877]
Predict...
[4.99507416 0.99803877]
Measured:  5
Update...
[4.99802295 0.99902266]


## Step 2 - Now, let's track Speedy in 2D (x and y)

The equations are similar, and we're still implementing the Constant Acceleration Model.
The only thing we're doing is adding a y dimension.

We will track position and velocity, so the state variable needs both of these. <p>The matrix formulation is:

$$ \mathbf x = \begin{bmatrix}x\\\dot x\\y\\\dot y\\\end{bmatrix}$$

So now we have to design our state transition. The Newtonian equations for a time step are:

$$\begin{aligned} x_t &= x_{t-1} + v\Delta t \\
 v_xt &= v_x{t-1}\end{aligned}$$
$$\begin{aligned} y_t &= y_{t-1} + v\Delta t \\
 v_yt &= v_y{t-1}\end{aligned}$$
 
Recall that we need to convert this into the linear equation

$$\begin{bmatrix}x\\\dot x\\y\\\dot y\\\end{bmatrix} = \mathbf F\begin{bmatrix}x\\\dot x\\y\\\dot y\\\end{bmatrix}$$

Setting

$$\mathbf F = \begin{bmatrix}1& \Delta t& 0& 0\\0& 1& 0& 0\\0& 0& 1& \Delta t\\ 0& 0& 0& 1\end{bmatrix}$$

gives us the equations above and also implements the **Constant Velocity Model**, but considering v_x and v_y being constant.

Finally, we design the measurement function. The measurement function needs to implement

$$\mathbf z = \mathbf{Hx}$$

Our sensor still only reads position, so it should take the position from the state, and 0 out the velocity and acceleration, like so:

$$\mathbf H =
\begin{bmatrix} 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0
\end{bmatrix}
$$

In [4]:
def TwoDimensionsKF(R_std, Q_std, dt):
    """ Create first order Kalman filter. 
    Specify R and Q as floats."""
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.x = np.zeros(4)
    kf.P = np.eye(4)*500
    kf.R = np.eye(2) * R_std**2
    q = Q_discrete_white_noise(dim=2, dt=dt, var=Q_std**2)
    kf.Q = block_diag(q, q)
    kf.F = np.array([[1, dt, 0,  0],
                    [0,  1, 0,  0],
                    [0,  0, 1, dt],
                    [0,  0, 0,  1]])
    kf.H = np.array([[1, 0, 0, 0],
                    [0, 0, 1, 0]])
    return kf

In [5]:
measurements = [(1,1), (2,2), (3,3), (4,4), (5,5)]
Q_std = 0.01
R_std = 5
dt = 1.

kf = TwoDimensionsKF(R_std, Q_std, dt)

for z in measurements:
    kf.predict()
    print("Predict...")
    print(kf.x)
    #print(kf.P)
    print("Measured: ",z)
    print("Update...")
    kf.update(z)
    print(kf.x)
    #print(kf.P)

Predict...
[0. 0. 0. 0.]
Measured:  (1, 1)
Update...
[0.97560976 0.48780491 0.97560976 0.48780491]
Predict...
[1.46341467 0.48780491 1.46341467 0.48780491]
Measured:  (2, 2)
Update...
[1.95933458 0.92421449 1.95933458 0.92421449]
Predict...
[2.88354907 0.92421449 2.88354907 0.92421449]
Measured:  (3, 3)
Update...
[2.97701572 0.97774543 2.97701572 0.97774543]
Predict...
[3.95476115 0.97774543 3.95476115 0.97774543]
Measured:  (4, 4)
Update...
[3.98571599 0.99066939 3.98571599 0.99066939]
Predict...
[4.97638538 0.99066939 4.97638538 0.99066939]
Measured:  (5, 5)
Update...
[4.99033074 0.99523621 4.99033074 0.99523621]


## Step 3 - Build Your Own 2D Kalman Filter
We are going to build a 2D filter that is going to track Speedy's x and y coordinates. 
Why?
I just want to make sure that you understood how the maths work and how to build a Kalman Filter for any project! That's why you'll implement the full filter by yourself!

The filter will implement the maths of Kalman Filters to track something in 2D. 2D means we track x and y. 
<p>We must define:<p>


*   x - Our state - It's a 4x1 matrix
*   P - Our covariance - It's a 4x4 matrix as well
*   Q - Our noise - It's a 4x4 matrix
*   F - Our transition matrix - It's a 4x4 matrix also
*   H - Our measurement transition matrix - It's a 2x4 matrix
*   R - Our measurement noise - It's a 4x4 matrix

Every time we measure something, we'll use a vector call Z.
*   Z - Our 2D Measurement - It's a 2x1 matrix
In the middle, we'll also use matrices such as the Kalman Gain or the Inverse matrix.
*   K - The Kalman Gain (should we trust the measurement or not) - It's a 4x2 matrix
*   y - The error - It's a 2x1 matrix
*   S - The system error - It's a 2x2 matrix
*   I - The identity matrix - It's a 4x4 matrix
*   SI - The Inverse of S - 2x2 as well

In [6]:
from numpy import eye, zeros, isscalar, dot
import numpy as np
from filterpy.common import pretty_str, reshape_z
from copy import deepcopy

In [7]:
class CustomKalmanFilter():
    def __init__(self, dim_u=0):
        self.x = np.zeros(4)
        self.F = np.array([[1, dt, 0,  0],
                    [0,  1, 0,  0],
                    [0,  0, 1, dt],
                    [0,  0, 0,  1]])
        self.P = np.eye(4)*500
        q = Q_discrete_white_noise(dim=2, dt=dt, var=Q_std**2)
        self.Q = block_diag(q, q)
        self.R = np.eye(2) * R_std**2
        self.y = np.zeros(2)
        self.z = np.zeros(2)
        self.H =np.array([[1, 0, 0, 0],
                          [0, 0, 1, 0]])
        self.I = np.eye(4)
        self.S = np.array([[0,0], [0,0]])
        self.SI = np.array([[0,0], [0,0]])
        self.K = np.array([[0,0,0,0],
                          [0,0,0,0]])

    def predict(self):
        x = dot(self.F,self.x)
        P = dot(dot(self.F,self.P), self.F.T) + self.Q
        self.x = x
        self.P = P

    def update(self, z):
        self.y = z - dot(self.H,self.x)
        self.S = dot(dot(self.H, self.P), self.H.T) + self.R
        self.SI = np.linalg.inv(self.S)
        self.K = dot(dot(self.P,self.H.T), self.SI)
        x = self.x + dot(self.K, self.y)
        P = dot(self.I - dot(self.K, self.H),self.P)
        self.x = x
        self.P = P

In [9]:
kf = CustomKalmanFilter()
kf.x = np.array([[0, 0, 0, 0]]).T
kf.P = np.eye(4) * 500.
dt = 1.0
kf.F = np.array([[1, dt, 0,  0],
                [0,  1, 0,  0],
                [0,  0, 1, dt],
                [0,  0, 0,  1]])

kf.H = np.array([[1, 0, 0, 0],
                    [0, 0, 1, 0]])
from scipy.linalg import block_diag
from filterpy.common import Q_discrete_white_noise
q = Q_discrete_white_noise(dim=2, dt=dt, var=0.001)
kf.Q = block_diag(q, q)
kf.R = np.array([[5., 0],
                      [0, 5]])

zs = [[1, 1], [2,2], [3,3], [4,4]]
for z in zs:
    kf.predict()
    print("Prediction...")
    print("Predicted X")
    print(kf.x)
    print("Predicted P")
    print(kf.P)
    kf.update(z)
    print("Measurement... z = ")
    print(z)
    print("Updated X")
    print(kf.x)
    print("Updated P")
    print(kf.P)

Prediction...
Predicted X
[[0.]
 [0.]
 [0.]
 [0.]]
Predicted P
[[1000.00025  500.0005     0.         0.     ]
 [ 500.0005   500.001      0.         0.     ]
 [   0.         0.      1000.00025  500.0005 ]
 [   0.         0.       500.0005   500.001  ]]
Measurement... z = 
[1, 1]
Updated X
[[0.99502488 0.99502488]
 [0.49751281 0.49751281]
 [0.99502488 0.99502488]
 [0.49751281 0.49751281]]
Updated P
[[  4.97512438   2.48756406   0.           0.        ]
 [  2.48756406 251.24434546   0.           0.        ]
 [  0.           0.           4.97512438   2.48756406]
 [  0.           0.           2.48756406 251.24434546]]
Prediction...
Predicted X
[[1.49253769 1.49253769]
 [0.49751281 0.49751281]
 [1.49253769 1.49253769]
 [0.49751281 0.49751281]]
Predicted P
[[261.19484796 253.73240952   0.           0.        ]
 [253.73240952 251.24534546   0.           0.        ]
 [  0.           0.         261.19484796 253.73240952]
 [  0.           0.         253.73240952 251.24534546]]
Measurement... z = 

In [10]:
class CustomKalmanFilter():
    def __init__(self,dim_x, dim_z, dim_u=0):
        self.dim_x = dim_x
        self.dim_z = dim_z
        self.dim_u = dim_u

        self.x = zeros((dim_x, 1))        # state
        self.P = eye(dim_x)               # uncertainty covariance
        self.Q = eye(dim_x)               # process uncertainty
        self.B = None                     # control transition matrix
        self.F = eye(dim_x)               # state transition matrix
        self.H = zeros((dim_z, dim_x))    # Measurement function
        self.R = eye(dim_z)               # state uncertainty
        self.M = np.zeros((dim_z, dim_z)) # process-measurement cross correlation
        self.z = np.array([[None]*self.dim_z]).T

        # gain and residual are computed during the innovation step. We
        # save them so that in case you want to inspect them for various
        # purposes
        self.K = np.zeros((dim_x, dim_z)) # kalman gain
        self.y = zeros((dim_z, 1))
        self.S = np.zeros((dim_z, dim_z)) # system uncertainty
        self.SI = np.zeros((dim_z, dim_z)) # inverse system uncertainty

        # identity matrix. Do not alter this.
        self._I = np.eye(dim_x)

        # these will always be a copy of x,P after predict() is called
        self.x_prior = self.x.copy()
        self.P_prior = self.P.copy()

        # these will always be a copy of x,P after update() is called
        self.x_post = self.x.copy()
        self.P_post = self.P.copy()

        self.inv = np.linalg.inv

    def predict(self, u=None, B=None, F=None, Q=None):
        if isscalar(Q):
            Q = eye(self.dim_x) * Q
        # x = Fx + Bu
        if B is not None and u is not None:
            self.x = dot(F, self.x) + dot(B, u)
        else:
            self.x = dot(F, self.x)

        # P = FPF' + Q
        self.P = dot(dot(F, self.P), F.T) + Q

        # save prior
        self.x_prior = self.x.copy()
        self.P_prior = self.P.copy()

    def update(self, z, R=None, H=None):
            z = reshape_z(z, self.dim_z, self.x.ndim)

            if isscalar(R):
                R = eye(self.dim_z) * R

            # y = z - Hx
            # error (residual) between measurement and prediction
            self.y = z - dot(H, self.x)

            # common subexpression for speed
            PHT = dot(self.P, H.T)

            # S = HPH' + R
            # project system uncertainty into measurement space
            self.S = dot(H, PHT) + R
            self.SI = self.inv(self.S)
            # K = PH'inv(S)
            # map system uncertainty into kalman gain
            self.K = dot(PHT, self.SI)

            # x = x + Ky
            # predict new x with residual scaled by the kalman gain
            self.x = self.x + dot(self.K, self.y)

            # P = (I-KH)P(I-KH)' + KRK'
            # This is more numerically stable
            # and works for non-optimal K vs the equation
            # P = (I-KH)P usually seen in the literature.

            I_KH = self._I - dot(self.K, H)
            self.P = dot(dot(I_KH, self.P), I_KH.T) + dot(dot(self.K, R), self.K.T)

            # save measurement and posterior state
            self.z = deepcopy(z)
            self.x_post = self.x.copy()
            self.P_post = self.P.copy()

# Part II - Implement the Constant Acceleration Model - Second Order Kalman Filter

So far, we have tracked Speedy's position and velocity. It was really great, but we assumed that it had a constant velocity. Sometimes, Speedy will accelerate and won't have a constant velocity; that's why we need to get one order higher and consider a constant acceleration.

No matter what we do, there is always an assumption to describe the movement.

What do I mean by order? <p>
An order is the number of derivatives required to model a system. 

*   Consider a system that doesn't change; such as the height of The Eiffel Tower. There is no change, so there is no need for a derivative, and the order of the system is zero. We could express this in an equation as $x = 324$.

*   Now, let's consider a tracking problem. A first order system has a first derivative. For example, change of position is velocity, and we can write this as $$ v = \frac{dx}{dt}$$

which we integrate into the Newtonian equation

$$ x = vt + x_0.$$

This is what we called the **constant velocity** model, because of the assumption of a constant velocity.

*   Finally, a second order system has a second derivative. The second derivative of position is acceleration, with the equation

$$a = \frac{d^2x}{dt^2}$$

which we integrate into 

$$ x = \frac{1}{2}at^2 +v_0t + x_0.$$

This is also known as the **constant acceleration** model.

```
# This is formatted as code
```

## Step 1 - Second Order Kalman Filter in 1D

A second order Kalman filter tracks a second order system, such as position, velocity and acceleration. The state variable will be

$$ \mathbf x = \begin{bmatrix}x\\\dot x\\\ddot{x}\end{bmatrix}$$

So now we have to design our state transition. The Newtonian equations for a time step are:

$$\begin{aligned} x_t &= x_{t-1} + v_{t-1}\Delta t + 0.5a_{t-1} \Delta t^2 \\
 v_t &= v_{t-1} + a_{t-1}\Delta t \\
 a_t &= a_{t-1}\end{aligned}$$
 
We'll have to implement this in matrix multiplication or linear equation.

$$\begin{bmatrix}x\\\dot x\\\ddot{x}\end{bmatrix} = \mathbf F\begin{bmatrix}x\\\dot x\\\ddot{x}\end{bmatrix}$$

Setting

$$\mathbf F = \begin{bmatrix}1 & \Delta t &.5\Delta t^2\\ 
0 & 1 & \Delta t \\
0 & 0 & 1\end{bmatrix}$$

gives us the equations above. 

Finally, we design the measurement function. The measurement function needs to implement

$$z = \mathbf{Hx}$$

Our sensor still only reads position, so it should take the position from the state, and 0 out the velocity, like so:

$$\mathbf H = \begin{bmatrix}1 & 0 & 0\end{bmatrix}$$

**This function constructs and returns a second order Kalman filter in 1D, only the position is tracked.**

In [12]:
def SecondOrderKF(R_std, Q_std, dt, P=100):
    """ Create second order Kalman filter. 
    Specify R and Q as floats."""
    
    kf = KalmanFilter(dim_x=3, dim_z=1)
    kf.x = np.zeros(3)
    kf.P = np.eye(3)*500
    kf.R = np.eye(1)*R_std**2
    kf.Q = Q_discrete_white_noise(3, dt, Q_std)
    kf.F = np.array([[1., dt, .5*dt*dt],
                     [0., 1.,       dt],
                     [0., 0.,       1.]])
    kf.H = np.array([[1., 0., 0.]])
    return kf

In [13]:
measurements = [1, 2, 3, 4, 6, 8, 10, 12]
Q_std = 0.01
R_std = 5
dt = 1.

kf = SecondOrderKF(R_std, Q_std, dt)
for z in measurements:
    kf.predict()
    print("Predict...")
    print(kf.x)
    #print(kf.P)
    print("Measured: ",z)
    print("Update...")
    kf.update(z)
    print(kf.x)
    #print(kf.P)

Predict...
[0. 0. 0.]
Measured:  1
Update...
[0.97826092 0.65217684 0.21739518]
Predict...
[1.73913535 0.86957202 0.21739518]
Measured:  2
Update...
[1.99376968 1.18380258 0.35825837]
Predict...
[3.35670144 1.54206095 0.35825837]
Measured:  3
Update...
[3.02399458 1.17913742 0.18021885]
Predict...
[4.29324142 1.35935627 0.18021885]
Measured:  4
Update...
[4.02955567 1.10753449 0.07391725]
Predict...
[5.17404879 1.18145174 0.07391725]
Measured:  6
Update...
[5.87877374 1.75019999 0.27213153]
Predict...
[7.76503949 2.02233152 0.27213153]
Measured:  8
Update...
[7.95307413 2.15121586 0.30979542]
Predict...
[10.2591877   2.46101128  0.30979542]
Measured:  10
Update...
[10.0654406  2.3464279  0.2810979]
Predict...
[12.55241745  2.6275258   0.2810979 ]
Measured:  12
Update...
[12.1667112   2.42746878  0.2372976 ]


### 2D 2nd Order Kalman Filter

State Vector: The state vector kf.x is a 6-dimensional vector representing [x_position, x_velocity, x_acceleration, y_position, y_velocity, y_acceleration].

State Covariance Matrix: kf.P is initialized with a value that reflects initial uncertainty about the state.

Measurement Noise: kf.R represents the measurement noise, assuming that only the positions in the x and y dimensions are measured.

Process Noise: kf.Q is set using Q_discrete_white_noise for a 3-dimensional state (position, velocity, acceleration) in both x and y components.

State Transition Matrix: kf.F is designed for a system with position, velocity, and acceleration. It accounts for the relationships between these variables over time.

Measurement Matrix: kf.H maps the state to the measurement space, set up to measure only the positions in both x and y dimensions.

In [21]:
def SecondOrderKF(R_std, Q_std, dt, P=100):
    """ Create second order Kalman filter. 
    Specify R and Q as floats."""
    
    kf = KalmanFilter(dim_x=6, dim_z=2)
    kf.x = np.zeros(6)
    kf.P = np.eye(6)*1000
    kf.R = np.eye(2)*R_std**2
    qt=Q_discrete_white_noise(dim=3,dt=dt,var=Q_std)
    kf.Q = block_diag(qt,qt)
    
    kf.F = np.array([[1, dt, 0.5*dt**2, 0, 0, 0],
                    [0, 1, dt, 0, 0, 0],
                    [0, 0, 1, 0, 0, 0],
                    [0, 0, 0, 1, dt, 0.5*dt**2],
                    [0, 0, 0, 0, 1, dt],
                    [0, 0, 0, 0, 0, 1]])# The only big problem will be here, you must write the equations by hand to make sure you didn't do any mistake
    kf.H = np.array([[1,0,0,0,0,0],
                    [0,0,0,1,0,0]])
    return kf

measurements = [(1,1), (2,2), (3,3), (4,4), (5,5)]
Q_std = 0.01
R_std = 5
dt = 1.

kf = SecondOrderKF(R_std, Q_std, dt)
for z in measurements:
    kf.predict()
    print("Predict...")
    print(kf.x)
    #print(kf.P)
    print("Measured: ",z)
    print("Update...")
    kf.update(z)
    print(kf.x)
    #print(kf.P)

Predict...
[0. 0. 0. 0. 0. 0.]
Measured:  (1, 1)
Update...
[0.989011   0.65934213 0.21978218 0.989011   0.65934213 0.21978218]
Predict...
[1.75824422 0.87912431 0.21978218 1.75824422 0.87912431 0.21978218]
Measured:  (2, 2)
Update...
[1.99696599 1.18052763 0.35581435 1.99696599 1.18052763 0.35581435]
Predict...
[3.3554008  1.53634198 0.35581435 3.3554008  1.53634198 0.35581435]
Measured:  (3, 3)
Update...
[3.01778923 1.13432613 0.13750754 3.01778923 1.13432613 0.13750754]
Predict...
[4.22086913 1.27183367 0.13750754 4.22086913 1.27183367 0.13750754]
Measured:  (4, 4)
Update...
[4.01806253 1.06655425 0.04628674 4.01806253 1.06655425 0.04628674]
Predict...
[5.10776015 1.11284099 0.04628674 5.10776015 1.11284099 0.04628674]
Measured:  (5, 5)
Update...
[5.01431996 1.03483976 0.01833423 5.01431996 1.03483976 0.01833423]


Each measurement z should contain the 2D position measurements (e.g., [x_position, y_position]). The filter will provide estimates for position, velocity, and acceleration in both x and y dimensions.
